# Chapter 9 トピック

In [1]:
import common_func as cf

## 9.1 潜在的意味インデキシング

In [3]:
# data/ch05にある各文書から文書ベクトルを作成する
book_texts = [cf.get_string_from_file('irpb-files/data/ch05/%d.txt' % i) for i in range(10)]
tfidf_model,dic,tfidf_weights = cf.get_tfidfmodel_and_weights(book_texts)


['パームアイランドはドバイの沖合に造られた人工島群で，ヤシの木をモチーフにデザインされています．この人工島には大型商業施設やホテル，別荘やなどが建てられています．\r\n', 'マービン・ミンスキーは，人工知能という分野の黎明期に活躍した研究者で，「人工知能の父」と呼ばれています．\r\n', 'イヌには，嘘をついたり，その場を取り繕うという行動が見られるという研究報告があります．これは，イヌの知能が高いことの証拠であると言われています．この他にも，イヌには推理能力があるといわれています．\r\n', '会話ボットと呼ばれる，Twitterなどで勝手につぶやいたり話しかけに対して適当に返事をするプログラムや，Siriなどの音声アシスタントもAIの一例であると考えられています．\r\n', '一般相対性理論により存在が予想されていたが，これまで目で見える形で確認することができなかったブラックホールを，地球規模の電波望遠鏡を使って撮影することに成功しました．\r\n', 'マッカーシーが1958年に人工知能の研究のために開発したLISPというプログラミング言語は，今では，お掃除ロボットのRoombaに使われています．\r\n', '「ニャンサムウェア」とは，感染するとファイルが暗号化されコンピュータが使用できなくなる「ランサムウェア」を捩ったもので，猫によるPCの作業妨害をいいます．\r\n', '新しい元号の「令和」の出典は，万葉集の「初春の令月にして，気淑よく風和やわらぎ，梅は鏡前の粉を披き，蘭は珮後の香を薫らす」であるとされています．\r\n', '人間の仕事の多くが人工知能に奪われてしまうという予測がある．実際，コールセンターに人工知能を導入した結果，人員が大幅に削減されたという事例がある．\r\n\r\n', 'ディープラーニングは脳の神経細胞ネットワークを模した機械学習の手法で，ネットワークを多層に構成にすることでデータの特徴をより深く学習させることができます．\r\n']


In [6]:
# LSIモデルの作成
from gensim.models import LsiModel

# トピックの数
num_topics = 5

# モデルの作成
lsi_model = LsiModel(corpus=tfidf_weights,id2word=dic,num_topics=num_topics)

In [7]:
# トピックの特徴語の表示
print(lsi_model.print_topic(0,3))

0.624*"知能" + 0.484*"人工" + 0.204*"マービン・ミンスキー"


In [11]:
#lsi_search関数
def lsi_search(texts,query,num_topics):
    
    from gensim.similarities import MatrixSimilarity
    
    # tfidfに基づいて語の重みを計算する
    tfidf_model,dic,text_tfidf_weights = cf.get_tfidfmodel_and_weights(texts)
    
    # LSIモデルを作成し、トピックの重みを計算する
    lsi_model = LsiModel(corpus=text_tfidf_weights,id2word=dic,num_topics=num_topics)
    lsi_weights = lsi_model[text_tfidf_weights]
    index = MatrixSimilarity(lsi_weights,num_features=len(dic))
    
    # queryのbag-of-wordsを作成し、重みを計算する
    query_bows = cf.get_bows([query],dic)
    query_tfidf_weights = cf.get_weights(query_bows,dic,tfidf_model)
    query_lsi_weights = lsi_model[query_tfidf_weights]
    
    # 類似度計算
    sims = index[query_lsi_weights[0]]
    
    # 類似度で降順にソートする
    return sorted(enumerate(sims),key=lambda x: x[1],reverse=True)

In [17]:
# vsm_searchによる検索(再掲)
from pprint import pprint

query = '人工知能'

# tfidfモデルに基づく検索(リスト5.2と同じ)
# 検索結果は関連殿高さで順位付けされている
tfidf_result = cf.vsm_search(book_texts,query)

# 文書番号と関連度の組を出力する
pprint(tfidf_result)

[(1, 0.7481151),
 (8, 0.6076249),
 (5, 0.31722325),
 (0, 0.21160641),
 (2, 0.18004589),
 (3, 0.0),
 (4, 0.0),
 (6, 0.0),
 (7, 0.0),
 (9, 0.0)]


In [18]:
# lsi_searchによる検索
num_topics = 5

query = '人工知能'

lsi_result = lsi_search(book_texts,query,num_topics)
pprint(lsi_result)

[(8, 0.99998426),
 (1, 0.99996907),
 (5, 0.9991018),
 (0, 0.5014957),
 (2, 0.40058395),
 (9, 0.0017561095),
 (3, 0.0),
 (6, 0.0),
 (7, 0.0),
 (4, -0.0029632207)]


In [19]:
# TF/IDFとLSIの検索性能比較

# '人工知能'をクエリとする検索の正解
right_answer = [0,1,0,1,0,1,0,0,1,1]

# ランキングの計算
tfidf_ranking = [x[0] for x in tfidf_result]
lsi_ranking = [x[0] for x in lsi_result]

# 平均適合率による検索性能評価の比較
print('TFIDF: %.4f' % cf.get_average_precision(tfidf_ranking,right_answer))
print('LSI:   %.4f' % cf.get_average_precision(lsi_ranking,right_answer))

TFIDF: 0.8211
LSI:   0.8648
